<a href="https://colab.research.google.com/github/timonpalm/TFCourse/blob/main/08_introduction_2_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in Tensorflow

# Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e9ca63ed-a264-3402-bb2f-6599f1efe7eb)


## Get Helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-01-26 19:18:33--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.3’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-01-26 19:18:33 (52.3 MB/s) - ‘helper_functions.py.3’ saved [10246/10246]



## Get a text dataset

Teh dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster)

See https://www.kaggle.com/competitions/nlp-getting-started/overview

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-01-26 19:18:40--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.207, 172.253.114.207, 172.253.119.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.3’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2024-01-26 19:18:40 (98.7 MB/s) - ‘nlp_getting_started.zip.3’ saved [607343/607343]



## Visualizing a text datset

In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
train_df.sample(n=5)

,id,keyword,location,text,target
5527,7883,quarantined,NaN,Reddit's new content policy goes into effect m...,0
2702,3876,detonation,Bangkok Thailand,A new favorite: Trivium - Detonation by @rrusa...,0
7014,10055,twister,NaN,Some curls come out so pretty and some look li...,0
3667,5221,fatality,Nairobi,Fatality!,0
5909,8437,sandstorm,Tractor land aka Bristol,I can't listen to Darude Sandstorm without exp...,0


In [11]:
import random
random_index = random.randint(0, len(train_df)-5)
for _, text, target in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  print(f"Target: {target}", "(real disaster)" if target > 0  else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
There have been zero spider bite-related fatalities in Australia since 1979.

---

Target: 1 (real disaster)
Text:
Wreckage Confirmed As Part of Missing Malaysia Airlines Flight MH370 http://t.co/yoPeYPJkb2 (VICE News)

---

Target: 1 (real disaster)
Text:
@MeyerBjoern @thelonevirologi @MackayIM of a major American newspaper running a series on all the alleged bioterrorism research going on 2/n

---

Target: 0 (not real disaster)
Text:
The courtÛªs reputation and prestige has sunk ever lower and Prosser who has served on the court since 1998 has... http://t.co/H09nYdbzoV

---

Target: 0 (not real disaster)
Text:
'Your love will surely come find us
Like blazing wild fires singing Your name'

---



### Split data into training and validation

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# Use train_test_split to split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [14]:
# Checl the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
# Check the first 10 sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Text Vectorization

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the deafult TextVerctorization
text_vectorizer = TextVectorization(max_tokens=10000, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int",
                                    output_sequence_length=None, # length of the sequences
                                    pad_to_max_tokens=True)

In [17]:
# Find the average number of tokens (words) in th training tweets
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [18]:
# Setup text vectorization varaibles
max_vocab_length = 10000 # max number of words in vocab
max_length = 15 # max length our sequences will be (how many words of the tweets will the model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [19]:
# Fit the text vectorizor to the train data
text_vectorizer.adapt(train_sentences)

In [20]:
# Create a sample sentence and tokinize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVecorized version:")
text_vectorizer([random_sentence])

Original text:
 Stay vigilent. Civil liberties are under constant attack. #nativehuman #myreligion  https://t.co/WWu070Tjej        

Vecorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 699, 7027, 2522,    1,   22,  204, 5931,  113,    1,    1,    1,
           0,    0,    0,    0]])>

In [22]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding Layer

In [23]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                              output_dim=128,
                              input_length=max_length)

embedding

In [24]:
# Get a radnom sentence from the training set
random_sentence = random.choice(train_sentences)

print(f"Original text:\n {random_sentence}\
        \n\nEmbedding version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Does the opening scene of Harry Potter and the Order of the Phoenix remind anyone else of the movie Twister? Just me? Okay        

Embedding version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.022724  ,  0.0263184 ,  0.02591031, ...,  0.00157546,
          0.03400469,  0.00920694],
        [-0.04069006, -0.02902658, -0.03839977, ..., -0.01076162,
         -0.00438211,  0.02401778],
        [-0.04554391, -0.01720723,  0.04522688, ..., -0.00213718,
         -0.03118942, -0.03885338],
        ...,
        [-0.04432576,  0.03054063,  0.00157443, ..., -0.00885032,
          0.00517534,  0.01401372],
        [-0.00230149, -0.00422196,  0.02071109, ...,  0.01375176,
          0.04462966, -0.00494741],
        [-0.03085362, -0.04719598,  0.02126427, ..., -0.00477303,
         -0.02396157, -0.04316865]]], dtype=float32)>

In [25]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.022724  ,  0.0263184 ,  0.02591031, -0.01669482,  0.02968761,
         0.00125699,  0.02794674,  0.0015109 , -0.00393447,  0.02526342,
         0.00642771,  0.00037215, -0.04161686, -0.04992262, -0.00681476,
        -0.0276917 , -0.03113775,  0.04855065, -0.03280897, -0.03603628,
         0.01482702,  0.02589868,  0.01776456, -0.01868583,  0.01043171,
        -0.00521391,  0.04576986, -0.02925644, -0.01776611,  0.00541846,
        -0.03860998, -0.0152458 , -0.01553835,  0.02700153,  0.03952474,
        -0.02645189,  0.04930799,  0.04079387, -0.01369124,  0.00567893,
        -0.00939541,  0.02817814,  0.03837272,  0.02076197,  0.0205769 ,
         0.00226267,  0.0149263 , -0.00642275,  0.00945573, -0.04838426,
        -0.03133618, -0.01163667,  0.03869972,  0.0293288 ,  0.02241016,
        -0.0423576 , -0.02350344,  0.0397276 , -0.03653059, -0.01845365,
         0.00346214,  0.04941556,  0.03743291, -0.04859905, -0.00262216,
  

## Modelling a text dataset (sunning a series of experiments)

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed Forward neural network (dense model)
* Model 2: LSTM (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

### Model 0: Getting a baseline

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [28]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

### Creating a evaluation function

In [29]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

def calculate_results(y_true, y_pred):
  res = {
      "accuracy": accuracy_score(y_true, y_pred),
      "precision": precision_score(y_true, y_pred, average="weighted"),
      "recall": recall_score(y_true, y_pred, average="weighted"),
      "f1": f1_score(y_true, y_pred, average="weighted")
  }

  return res

In [30]:
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: Feed Forward

In [31]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dim strings
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs=inputs, outputs=outputs, name="model_1_dense")

In [32]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [33]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [34]:
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 15s 52ms/step - loss: 0.6099 - accuracy: 0.6895 - val_loss: 0.5378 - val_accuracy: 0.7467
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.4409 - accuracy: 0.8175 - val_loss: 0.4711 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3459 - accuracy: 0.8606 - val_loss: 0.4626 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2836 - accuracy: 0.8883 - val_loss: 0.4643 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2369 - accuracy: 0.9133 - val_loss: 0.4837 - val_accuracy: 0.7848


In [35]:
preds = model_1.predict(val_sentences)
preds.shape

24/24 [==============================] - 0s 4ms/step


(762, 1)

In [36]:
preds = tf.squeeze(tf.round(preds))

In [37]:
model_1_results = calculate_results(val_labels, preds)
model_1_results

{'accuracy': 0.7847769028871391,
 'precision': 0.7900209877855079,
 'recall': 0.7847769028871391,
 'f1': 0.781600538849599}

## Visualizing learned embeddings

In [38]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [40]:
# Get the weight matrix of embedding layer
# (these are the numerical respresentations of each token in our training data)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

Use tensorflow projector [tool](https://projector.tensorflow.org/) to visualize the embeddings. Find the code [here](https://www.tensorflow.org/text/guide/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk)

In [41]:
# import io

# out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
# out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

# for index, word in enumerate(words_in_vocab):
#   if index == 0:
#     continue  # skip 0, it's padding.
#   vec = embed_weights[index]
#   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#   out_m.write(word + "\n")
# out_v.close()
# out_m.close()

In [42]:
# # download files to upload to projector
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

## Recurrent Neural Networks (RNN)

### Model 2: LSTM

In [43]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(units=64, return_sequences=True)(x) # when you'are stacking LSTM cells together you need the sequences
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name="model_2_lstm")

In [44]:
model_2.summary()

Model: "model_2_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                      

In [45]:
# compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [46]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 14s 43ms/step - loss: 0.2177 - accuracy: 0.9261 - val_loss: 0.5283 - val_accuracy: 0.7900
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1579 - accuracy: 0.9422 - val_loss: 0.6242 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1285 - accuracy: 0.9502 - val_loss: 0.7513 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1041 - accuracy: 0.9590 - val_loss: 0.7948 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0813 - accuracy: 0.9677 - val_loss: 0.9211 - val_accuracy: 0.7743


In [47]:
pred_probs = model_2.predict(val_sentences)
pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[2.1974100e-02],
       [6.7327946e-01],
       [9.9993300e-01],
       [1.2089860e-01],
       [3.3438823e-04],
       [9.9840999e-01],
       [8.0099410e-01],
       [9.9996603e-01],
       [9.9989021e-01],
       [6.9942147e-01]], dtype=float32)

In [48]:
pred = tf.squeeze(tf.round(pred_probs))
model_2_results = calculate_results(val_labels, preds)
model_2_results

{'accuracy': 0.7847769028871391,
 'precision': 0.7900209877855079,
 'recall': 0.7847769028871391,
 'f1': 0.781600538849599}

### Model 3: GRU Network

In [49]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(units=64, return_sequences=True)(x) # when you'are stacking LSTM cells together you need the sequences
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs, outputs, name="model_3_lstm")

In [50]:
# compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [51]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 13s 45ms/step - loss: 0.1529 - accuracy: 0.9492 - val_loss: 0.7028 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0855 - accuracy: 0.9695 - val_loss: 0.8131 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0684 - accuracy: 0.9724 - val_loss: 1.1375 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0573 - accuracy: 0.9745 - val_loss: 0.9307 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0555 - accuracy: 0.9756 - val_loss: 1.3520 - val_accuracy: 0.7769


In [52]:
pred_probs = model_3.predict(val_sentences)
pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[1.7173984e-03],
       [5.9571344e-01],
       [9.9995399e-01],
       [2.2056237e-01],
       [3.3495031e-05],
       [9.9993098e-01],
       [9.9461138e-01],
       [9.9998033e-01],
       [9.9997318e-01],
       [7.0902836e-01]], dtype=float32)

In [53]:
pred = tf.squeeze(tf.round(pred_probs))
model_3_results = calculate_results(val_labels, preds)
model_3_results

{'accuracy': 0.7847769028871391,
 'precision': 0.7900209877855079,
 'recall': 0.7847769028871391,
 'f1': 0.781600538849599}

### Model 4: Bidirectional LSTM

In [54]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs=inputs, outputs=outputs, name="model_4_bidirectional")

In [55]:
# compile the model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [56]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 19s 56ms/step - loss: 0.1028 - accuracy: 0.9676 - val_loss: 1.2212 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0482 - accuracy: 0.9772 - val_loss: 1.5572 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0450 - accuracy: 0.9785 - val_loss: 1.5570 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0445 - accuracy: 0.9785 - val_loss: 1.3120 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0439 - accuracy: 0.9796 - val_loss: 1.5880 - val_accuracy: 0.7664


In [57]:
pred_probs = model_4.predict(val_sentences)
pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[4.9174333e-01],
       [8.7370622e-01],
       [9.9997997e-01],
       [6.2444728e-02],
       [1.6546865e-05],
       [9.9991512e-01],
       [5.6287473e-01],
       [9.9999356e-01],
       [9.9998736e-01],
       [8.8701320e-01]], dtype=float32)

In [58]:
pred = tf.squeeze(tf.round(pred_probs))
model_4_results = calculate_results(val_labels, preds)
model_4_results

{'accuracy': 0.7847769028871391,
 'precision': 0.7900209877855079,
 'recall': 0.7847769028871391,
 'f1': 0.781600538849599}

## Model 5: Conv1D




In [59]:
embedding_text = embedding(text_vectorizer(["This is text sentence"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation="relu",
                        padding="valid")

conv_1d_output = conv_1d(embedding_text)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_text.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [60]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=32,
                  kernel_size=5,
                  activation="relu",
                  padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,
                      activation="sigmoid")(x)

model_5 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [61]:
model_5.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=["accuracy"])

model_5.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 9s 36ms/step - loss: 0.1378 - accuracy: 0.9558 - val_loss: 0.8334 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0768 - accuracy: 0.9712 - val_loss: 0.9874 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0631 - accuracy: 0.9759 - val_loss: 1.0931 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0554 - accuracy: 0.9765 - val_loss: 1.1254 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0497 - accuracy: 0.9790 - val_loss: 1.1430 - val_accuracy: 0.7651


In [62]:
pred_probs = model_5.predict(val_sentences)
pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[9.1051537e-01],
       [8.5071701e-01],
       [9.9996507e-01],
       [2.0891041e-02],
       [2.4677857e-07],
       [9.9511826e-01],
       [9.9190217e-01],
       [9.9997938e-01],
       [9.9999928e-01],
       [7.2712034e-01]], dtype=float32)

In [63]:
pred = tf.squeeze(tf.round(pred_probs))
model_5_results = calculate_results(val_labels, preds)
model_5_results

{'accuracy': 0.7847769028871391,
 'precision': 0.7900209877855079,
 'recall': 0.7847769028871391,
 'f1': 0.781600538849599}

## Model 6: Pretrained Encoder

In [64]:
import tensorflow_hub as hub

embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2")
embed_samples = embed([sample_sentence,
                       "When you can use the universal sentence encoder on a sentence, it turns it into numbers"])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [65]:
embed_samples[0].shape

TensorShape([512])

In [66]:
sentence_encoder_layer = hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",
                                        input_shape=[],
                                        dtype=tf.string,
                                        name="USE")

model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

In [67]:
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_8 (Dense)             (None, 64)                32832     
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [68]:
model_6.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 6s 14ms/step - loss: 0.5013 - accuracy: 0.7865 - val_loss: 0.4477 - val_accuracy: 0.7966
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4137 - accuracy: 0.8143 - val_loss: 0.4382 - val_accuracy: 0.8084
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4000 - accuracy: 0.8218 - val_loss: 0.4326 - val_accuracy: 0.8097
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3926 - accuracy: 0.8260 - val_loss: 0.4294 - val_accuracy: 0.8071
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3860 - accuracy: 0.8302 - val_loss: 0.4277 - val_accuracy: 0.8097


In [69]:
pred_probs = model_6.predict(val_sentences)
pred_probs[:10]

24/24 [==============================] - 1s 10ms/step


array([[0.15907067],
       [0.82316285],
       [0.9906437 ],
       [0.2084867 ],
       [0.7553013 ],
       [0.78500503],
       [0.98686045],
       [0.98387086],
       [0.9479458 ],
       [0.10022128]], dtype=float32)

In [70]:
pred = tf.squeeze(tf.round(pred_probs))
model_6_results = calculate_results(val_labels, preds)
model_6_results

{'accuracy': 0.7847769028871391,
 'precision': 0.7900209877855079,
 'recall': 0.7847769028871391,
 'f1': 0.781600538849599}

## Model 7: TF Hub Pretrained USE but with 10% of the training data

In [71]:
# Create subsets of 10% of the data
train_10_percent = train_df[["text", "target"]].sample(frac=0.1, random_state=42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()

In [72]:
# check the number of targets in our subset of data
train_10_percent["target"].value_counts()

0    425
1    336
Name: target, dtype: int64

In [73]:
train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [74]:
sentence_encoder_layer = hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",
                                        input_shape=[],
                                        dtype=tf.string,
                                        name="USE")

model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

In [75]:
model_7.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=["accuracy"])

model_7.fit(train_sentences_10_percent,
            train_labels_10_percent,
            epochs=5,
            steps_per_epoch=len(train_sentences_10_percent),
            validation_data=(val_sentences, val_labels))

Epoch 1/5
761/761 [==============================] - 10s 9ms/step - loss: 0.5383 - accuracy: 0.7451 - val_loss: 0.4515 - val_accuracy: 0.7835
Epoch 2/5
761/761 [==============================] - 7s 10ms/step - loss: 0.3818 - accuracy: 0.8331 - val_loss: 0.4498 - val_accuracy: 0.7940
Epoch 3/5
761/761 [==============================] - 6s 8ms/step - loss: 0.3302 - accuracy: 0.8555 - val_loss: 0.4583 - val_accuracy: 0.7992
Epoch 4/5
761/761 [==============================] - 7s 9ms/step - loss: 0.2789 - accuracy: 0.8883 - val_loss: 0.4785 - val_accuracy: 0.7940
Epoch 5/5
761/761 [==============================] - 7s 10ms/step - loss: 0.2372 - accuracy: 0.9014 - val_loss: 0.4935 - val_accuracy: 0.7940


In [76]:
pred_probs = model_7.predict(val_sentences)
pred_probs[:10]

24/24 [==============================] - 1s 9ms/step


array([[0.00766701],
       [0.8824376 ],
       [0.9993241 ],
       [0.5733831 ],
       [0.75307566],
       [0.5154333 ],
       [0.9824255 ],
       [0.99722075],
       [0.78739876],
       [0.04954262]], dtype=float32)

In [77]:
pred = tf.squeeze(tf.round(pred_probs))
model_7_results = calculate_results(val_labels, preds)
model_7_results

{'accuracy': 0.7847769028871391,
 'precision': 0.7900209877855079,
 'recall': 0.7847769028871391,
 'f1': 0.781600538849599}

# Comparing the performance of each of our models

In [78]:
# combine model results into dataframe
all_model_results = pd.DataFrame({"0_baseline": baseline_results,
                                  "1_simple_dense": model_1_results,
                                  "2_lstm": model_2_results,
                                  "3_gru": model_3_results,
                                  "4_bidirectional": model_4_results,
                                  "5_conv1d": model_5_results,
                                  "6_tf_hub_use_encoder": model_6_results,
                                  "7_tf_hub_use_encoder_10_percent": model_7_results})

all_model_results.transpose()

,accuracy,precision,recall,f1
0_baseline,0.792651,0.811139,0.792651,0.786219
1_simple_dense,0.784777,0.790021,0.784777,0.781601
2_lstm,0.784777,0.790021,0.784777,0.781601
3_gru,0.784777,0.790021,0.784777,0.781601
4_bidirectional,0.784777,0.790021,0.784777,0.781601
5_conv1d,0.784777,0.790021,0.784777,0.781601
6_tf_hub_use_encoder,0.784777,0.790021,0.784777,0.781601
7_tf_hub_use_encoder_10_percent,0.784777,0.790021,0.784777,0.781601
